In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from torch.utils.data import WeightedRandomSampler
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.nn import CosineSimilarity
from torchvision.transforms import ToTensor
from torchvision import transforms
import torchvision.models as models
from pathlib import Path
import PIL.Image
import random
import math
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ExponentialLR
import os
import glob
import cv2
import numpy as np
import PIL
from sklearn.metrics import cohen_kappa_score

# Copy pretrained models

In [ ]:
# copy pretrained weights for resnet50 to the folder fastai will search by default
Path('/tmp/.cache/torch/checkpoints/').mkdir(exist_ok=True, parents=True)
!cp /kaggle/input/fastai-pretrained-models/* /tmp/.cache/torch/checkpoints/
!cp /kaggle/input/pretrained-pytorch-models/* /tmp/.cache/torch/checkpoints/
!cp /kaggle/input/resnet152/* /tmp/.cache/torch/checkpoints/
!mv /tmp/.cache/torch/checkpoints/resnet152.pth /tmp/.cache/torch/checkpoints/resnet152-b121ed2d.pth
!ls /tmp/.cache/torch/checkpoints/

# Set fixed seed

In [ ]:
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

# Data from 2015

In [ ]:
path = Path('../input/resized-2015-2019-blindness-detection-images/')

In [ ]:
img_paths = []
img_diags = []
img_ids = []

In [ ]:
df = pd.read_csv(path/'labels/testLabels15.csv')
df['path'] = df.image.apply(lambda p: path/'resized test 15'/(p + '.jpg'))
img_paths.extend(df['path'].tolist())
img_diags.extend(df['level'].tolist())
img_ids.extend(df['image'].tolist())

In [ ]:
df = pd.read_csv(path/'labels/trainLabels15.csv')
df['path'] = df.image.apply(lambda p: path/'resized train 15'/(p + '.jpg'))
img_paths.extend(df['path'].tolist())
img_diags.extend(df['level'].tolist())
img_ids.extend(df['image'].tolist())

In [ ]:
tr_df = pd.DataFrame({'id_code':img_ids, 'diagnosis': img_diags, 'path':img_paths})

In [ ]:
tr_df, tr_va_df = train_test_split(tr_df, test_size=0.05, random_state=42, stratify=tr_df.diagnosis)

# Data from 2019

In [ ]:
# path = Path('../input/aptos2019-blindness-detection')
path = Path('../input/resized-2015-2019-blindness-detection-images/')

In [ ]:
df = pd.read_csv(path/'labels/trainLabels19.csv')

In [ ]:
img_paths = []
img_diags = []
img_ids = []

In [ ]:
df['path'] = df.id_code.apply(lambda p: path/'resized train 19'/(p + '.jpg'))
img_paths.extend(df['path'].tolist())
img_diags.extend(df['diagnosis'].tolist())
img_ids.extend(df['id_code'].tolist())

In [ ]:
te_va_df = pd.DataFrame({'id_code':img_ids, 'diagnosis': img_diags, 'path':img_paths})
tr_df = pd.DataFrame({'id_code':img_ids, 'diagnosis': img_diags, 'path':img_paths})

In [ ]:
tr_df, tr_va_df = train_test_split(tr_df, test_size=0.20, random_state=42, stratify=tr_df.diagnosis)

# Circle crop

In [ ]:
class CircleCrop:
    def __init__(self, radius):
        self.radius = radius
        circle_img = np.zeros((2 * radius, 2 * radius), np.uint8)
        cv2.circle(circle_img, (radius, radius), int(radius), 1, thickness=-1)
        self.mask = torch.tensor(circle_img, dtype=torch.float32)
        
    def __call__(self, img):
        return torch.mul(img, self.mask)
    
    def __repr__(self):
        return self.__class__.__name__ + '(radius={0})'.format(self.radius)

# APTOS Dataset

In [ ]:
class APTOSDataset(Dataset):
    def __init__(self, df, labels=None, augmentation=False, img_size=480):
        self.len = len(df)
        self.labels = labels
        self.images = df.path.tolist()
        self.trfms = transforms.Compose([
            transforms.Resize(img_size),
            transforms.CenterCrop(img_size),
            transforms.RandomApply((
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomAffine(degrees=180, scale=(1.0, 1.5), resample=PIL.Image.BICUBIC),
                ), p = 1 if augmentation else 0  
            ),
            transforms.ColorJitter(
                saturation=(1.0, 1.0),
                brightness=(1.0, 1.5), 
                contrast=(1.5, 2.5),
            ),
            transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225]),
            CircleCrop(img_size // 2),
            ])
        
    def __getitem__(self, index):
        if self.labels is not None:
            label = self.labels[index]
        else:
            label = 0
        path = self.images[index]
        img = PIL.Image.open(path)
        img = self.trfms(img)
        return img, label
    
    def __len__(self):
        return self.len

# Create datasets

In [ ]:
path = Path('../input')
img_size = 256

In [ ]:
tr_ds = APTOSDataset(tr_df, tr_df['diagnosis'].values, augmentation=True, img_size=img_size)

In [ ]:
tr_va_ds = APTOSDataset(tr_va_df, tr_va_df['diagnosis'].values, augmentation=False, img_size=img_size)

In [ ]:
te_va_ds = APTOSDataset(te_va_df, te_va_df['diagnosis'].values, augmentation=False, img_size=img_size)

In [ ]:
len(tr_ds), len(tr_va_ds), len(te_va_ds)

In [ ]:
img, label = tr_ds[random.randint(0, len(tr_ds) - 1)]
plt.imshow(transforms.ToPILImage()(img))

In [ ]:
img, label = tr_va_ds[random.randint(0, len(tr_va_ds) - 1)]
plt.imshow(transforms.ToPILImage()(img))

In [ ]:
img, label = te_va_ds[random.randint(0, len(te_va_ds) - 1)]
plt.imshow(transforms.ToPILImage()(img))

# Dataloaders

In [ ]:
bs = 48
nw = 4

In [ ]:
# class_weights = torch.tensor([0.05,0.20,0.15,0.30,0.30])
# tr_labels = tr_ds.labels
# sample_weights = class_weights[tr_labels]
# sampler = WeightedRandomSampler(sample_weights, num_samples=len(tr_ds))

In [ ]:
tr_dl = DataLoader(tr_ds, batch_size=bs, num_workers=nw, drop_last=True, pin_memory=True, shuffle=True)
# tr_dl = DataLoader(tr_ds, batch_size=bs, num_workers=nw, drop_last=False, pin_memory=True, sampler=sampler)

In [ ]:
tr_va_dl = DataLoader(tr_va_ds, batch_size=bs, num_workers=nw, drop_last=False, pin_memory=True, shuffle=False)

In [ ]:
te_va_dl = DataLoader(te_va_ds, batch_size=bs, num_workers=nw, drop_last=False, pin_memory=True, shuffle=False)

# Model

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

In [ ]:
class Bias(nn.Module):
    def __init__(self, num_out):
        super(Bias, self).__init__()
        self.bias = nn.Parameter(torch.zeros(num_out).float())
        
    def forward(self, x):
        return x + self.bias

In [ ]:
num_classes = 5

In [ ]:
# # Linear classifier
# model = nn.Sequential(
# #     nn.AvgPool2d(kernel_size=2),
#     Flatten(),
#     nn.Linear(in_features=480 * 480 * 3,
#              out_features=num_classes)
# ).cuda()

In [ ]:
model = models.densenet201(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.classifier = nn.Sequential(
    nn.BatchNorm1d(1920),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=1920, out_features=512),
    nn.ReLU(inplace=True),
    nn.BatchNorm1d(512),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=512, out_features=num_classes),
    nn.Linear(in_features=num_classes, out_features=1, bias=False),
    Bias(num_classes - 1)
#     nn.Hardtanh(min_val=0.0, max_val=4.0)
)
# model.fc = nn.Linear(in_features=2048, out_features=num_classes)
# model.classifier.requires_grad = True
model = model.cuda()

In [ ]:
# for name, param in model.named_parameters():
# # for layer in model.features:
#     if 'norm' in name:
# #     if isinstance(layer, nn.BatchNorm2d):
# #         for param in layer.parameters():
#         param.requires_grad = True
#     else:
# #         for param in layer.parameters():
#         param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = False
    
# for layer in model.modules():
#     if isinstance(layer, nn.BatchNorm1d):
#         for param in layer.parameters():
#             param.requires_grad = False
    
# for param in model.features.transition1.parameters():
#     param.requires_grad = True
    
# for param in model.features.transition2.parameters():
#     param.requires_grad = True
    
# for param in model.features.transition3.parameters():
#     param.requires_grad = True

In [ ]:
for name, param in model.named_parameters():
    param.requires_grad = True

In [ ]:
# model.classifier[8].bias.requires_grad = True

In [ ]:
for name, param in model.named_parameters():
    print('{:<48}\t{}\t{}'.format(name, param.requires_grad, torch.numel(param)))

In [ ]:
optimizer = Adam(model.parameters(), lr=3e-5)

In [ ]:
checkpoint = torch.load('../input/aptos-pytorch/checkpoint.tar')
model.load_state_dict(checkpoint['model'], strict=False)
optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
# for p in model.layer4.parameters():
#     p.requires_grad = True
# for p in model.layer3.parameters():
#     p.requires_grad = True

In [ ]:
optimizer.param_groups[0]['lr'] = 3e-6

In [ ]:
# optimizer.param_groups[0]['betas'] = (0.999, 0.999)

# Loss functions

In [ ]:
class CumulativeProbabilityOrdinalLoss(nn.Module):
    def __init__(self):
        super(CumulativeProbabilityOrdinalLoss, self).__init__()
        
    def forward(self, output, label):
        prob = F.softmax(output, dim=1)
        cdf = torch.cumsum(prob, dim=1)
        one_hot = torch.zeros_like(prob)
        one_hot.scatter_(1, label.view(-1, 1), 1)
        label_cdf = torch.cumsum(one_hot, dim=1)
        return F.mse_loss(cdf, label_cdf)

In [ ]:
class CoralLoss(nn.Module):
    def __init__(self):
        super(CoralLoss, self).__init__()
        self.exceed_rank = torch.tensor([[0, 0, 0, 0],
                            [1, 0, 0, 0],
                            [1, 1, 0, 0],
                            [1, 1, 1, 0],
                            [1, 1, 1, 1],
                           ]).float().cuda()
        
    def forward(self, output, label):
        return F.binary_cross_entropy_with_logits(output, self.exceed_rank[label])

In [ ]:
class SquareWeightedMSEOnProbabilityLoss(nn.Module):
    def __init__(self):
        super(SquareWeightedMSEOnProbabilityLoss, self).__init__()
        self.weights = torch.tensor([
            [0, 1, 4, 9, 16], 
            [1, 0, 1, 4, 9], 
            [4, 1, 0, 1, 4], 
            [9, 4, 1, 0, 1], 
            [16, 9, 4, 1, 0]], dtype=torch.float32).cuda()
        
    def forward(self, output, label):
        prob = F.softmax(output, dim=1)
        return torch.mean(torch.pow(torch.mul(self.weights[label], prob), 2))

In [ ]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=3., reduction='mean', weight=None):
        super().__init__()
        self.gamma = gamma
        self.reduction = reduction
        self.weight = weight

    def forward(self, inputs, targets):
        CE_loss = nn.CrossEntropyLoss(reduction='none', weight=self.weight)(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = ((1 - pt)**self.gamma) * CE_loss
        if self.reduction == 'sum':
            return F_loss.sum()
        elif self.reduction == 'mean':
            return F_loss.mean()

In [ ]:
# criterion = nn.CrossEntropyLoss()
# criterion = LabelSmoothingLoss(0.2)
# ce_loss_weights = torch.tensor([1.0, 15.0, 5.0, 10.0, 10.0])
# ce_loss_weights /= ce_loss_weights.sum()
# criterion = FocalLoss()
# criterion = nn.MSELoss()
criterion = CoralLoss()
# criterion = CumulativeProbabilityOrdinalLoss()
# criterion = SquareWeightedMSEOnProbabilityLoss()

# Train

In [ ]:
def reg_out_to_class(out):
    return torch.round(out).int()

In [ ]:
def coral_out_to_class(out):
    out = torch.sigmoid(out)
    out = out > 0.5
    return torch.sum(out, dim=1)

In [ ]:
def train_one_epoch(model, optimizer, criterion, dl, batch_count=None):
    if batch_count is None:
        batch_count = len(dl)
    metrics = []
#     count = 0
    for inputs, targets in tqdm(dl):
        model.train()
        inputs, targets = inputs.cuda(), targets.cuda() #targets.reshape((-1,1)).float().cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            model.eval()
#             preds = torch.argmax(outputs, dim=1)
#             preds = reg_out_to_class(outputs)
            preds = coral_out_to_class(outputs)
#             print(preds.dtype)
#             targets = targets.int()
            accu = (preds == targets).sum().item() / float(inputs.shape[0])
            metrics.append((loss.item(), accu))
#         count += 1
#         if count >= batch_count:
#             break
    
    return metrics

In [ ]:
def validate_model_on(model, dl, criterion):
    with torch.no_grad():
        model.eval()
        conf_matrix = [[0 for i in range(num_classes)] for j in range(num_classes)]
        loss = 0.0
        accu = 0.0
        y1, y2 = [], []
        for inputs, targets in tqdm(dl):
            inputs, targets = inputs.cuda(), targets.cuda() #targets.reshape((-1,1)).float().cuda()
            outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             preds = reg_out_to_class(outputs)
            preds = coral_out_to_class(outputs)
            y1.extend(targets.tolist())
            y2.extend(preds.tolist())
            loss += criterion(outputs, targets).item()
#             targets = targets.int()
#             print(preds, targets)
            accu += (preds == targets).sum().item() / float(inputs.shape[0])
            for i in range(len(preds)):
                conf_matrix[targets[i]][preds[i]] += 1
        loss /= len(dl)
        accu /= len(dl)
        kappa = cohen_kappa_score(y1, y2, weights='quadratic')
    return loss, accu, kappa, conf_matrix

In [ ]:
num_epochs = 40

In [ ]:
tr_metrics = []
tr_va_metrics = []
te_va_metrics = []

In [ ]:
for epoch in tqdm(range(num_epochs)):                        
    print('Epoch: {}'.format(epoch))
    metrics = train_one_epoch(model, optimizer, criterion, tr_dl)
    mean = lambda l: sum(l) / len(l)
    tr_loss = mean(list(map(lambda m:m[0], metrics)))
    tr_accu = mean(list(map(lambda m:m[1], metrics)))
    tr_metrics.extend(metrics)
    print('Train Loss: {} Acc: {}'.format(tr_loss, tr_accu))

    tr_va_loss, tr_va_accu, tr_va_kappa, tr_va_conf_matrix = validate_model_on(model, tr_va_dl, criterion)
    print('Train Val Loss: {} Acc: {} Kappa: {}'.format(tr_va_loss, tr_va_accu, tr_va_kappa))
    for row in tr_va_conf_matrix:
        print(row)
    tr_va_metrics.append((tr_va_loss, tr_va_accu, tr_va_kappa))
    
#     te_va_loss, te_va_accu, te_va_kappa, te_va_conf_matrix = validate_model_on(model, te_va_dl, criterion)
#     print('Test Val Loss: {} Acc: {} Kappa: {}'.format(te_va_loss, te_va_accu, te_va_kappa))
#     for row in te_va_conf_matrix:
#         print(row)
#     te_va_metrics.append((te_va_loss, te_va_accu, te_va_kappa))

In [ ]:
# del model
# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

In [ ]:
plt.plot(list(map(lambda t:t[0], tr_metrics)))

In [ ]:
plt.plot(list(map(lambda t:t[1], tr_metrics)))

In [ ]:
plt.plot(list(map(lambda t:t[0], tr_va_metrics)))
plt.plot(list(map(lambda t:t[0], te_va_metrics)))

In [ ]:
plt.plot(list(map(lambda t:t[1], tr_va_metrics)))
plt.plot(list(map(lambda t:t[1], te_va_metrics)))

In [ ]:
plt.plot(list(map(lambda t:t[2], tr_va_metrics)))
plt.plot(list(map(lambda t:t[2], te_va_metrics)))

In [ ]:
torch.save({'model': model.state_dict(),
           'optimizer': optimizer.state_dict()}, '/kaggle/working/checkpoint.tar')